In [1]:
import pandas as pd

In [2]:
from swat import *
import dlpy
from dlpy.timeseries import plot_timeseries
from dlpy.splitting import two_way_split
from dlpy import Sequential
from dlpy.layers import * 
from dlpy.model import Optimizer, AdamSolver, Sequence
from dlpy.metrics import accuracy_score, roc_auc_score, confusion_matrix
from getpass import getpass

# Connect to SAS  Viya

In [3]:
sas = CAS('localhost', 5570, 'sasdemo', getpass('Please enter your passwod:'))

Please enter your passwod:········


In [4]:
sas.loadactionset('fedSql')
print('Done!')

NOTE: Added action set 'fedSql'.
Done!


# Test Vars

In [ ]:
test_vars = sas.read_csv('/sasdata/pdm/test_vars.csv', casout='test_vars')

# ESP Vars

In [ ]:
esp_vars = sas.read_csv('/sasdata/pdm/stream_out_vars.csv', casout='esp_vars')
esp_vars.shape

# Compare Vars

In [ ]:
sas.fedsql.execdirect(
    '''
    create table t1 {options replace=true} as select
        a."id",
        a."cycle",
        a."setting1" as "a_setting1",
        b."setting1" as "b_setting1",
        a."setting1_lag49" as "a_setting1_lag49",
        b."setting1_lag49" as "b_setting1_lag49",
        a."s2" as "a_s2",
        b."s2" as "b_s2",
        a."s2_lag49" as "a_s2_lag49",
        b."s2_lag49" as "b_s2_lag49",
        a."age" as "a_age",
        b."age" as "b_age",
        a."age_lag49" as "a_age_lag49",
        b."age_lag49" as "b_age_lag49",
        a."nearfailure" as "a_nearfailure",
        b."nearfailure" as "b_nearfailure",
        a."nearfailure_lag49" as "a_nearfailure_lag49",
        b."nearfailure_lag49" as "b_nearfailure_lag49"
    from test_vars as a
    inner join esp_vars as b
    on
        a."id" = b."id" and
        a."cycle" = b."cycle"
    '''
)
t1 = sas.CASTable('t1')

In [ ]:
t1.head(200)

In [ ]:
sas.fedsql.execdirect(
    '''
    create table t2 {options replace=true} as select
        sum(case when a."setting1" <> b."setting1" then 1 else 0 end) as "setting1",
        sum(case when a."setting1_lag49" <> b."setting1_lag49" then 1 else 0 end) as "setting1_lag49",
        sum(case when a."s2" <> b."s2" then 1 else 0 end) as "s2",
        sum(case when a."s2_lag49" <> b."s2_lag49" then 1 else 0 end) as "s2_lag49",
        sum(case when a."age" <> b."age" then 1 else 0 end) as "age",
        sum(case when a."age_lag49" <> b."age_lag49" then 1 else 0 end) as "age_lag49",
        sum(case when a."nearfailure" <> b."nearfailure" then 1 else 0 end) as "nearfailure",
        sum(case when a."nearfailure_lag49" <> b."nearfailure_lag49" then 1 else 0 end) as "nearfailure_lag49"
    from test_vars as a
    inner join esp_vars as b
    on
        a."id" = b."id" and
        a."cycle" = b."cycle"
    '''
)
t2 = sas.CASTable('t2')

In [ ]:
t2.head()

# Test Scores

In [11]:
test1_scores = sas.read_csv('/sasdata/pdm/lstm_002_preds.csv',
                           casout=CASTable(name='test1_scores', replace=True))
print(test1_scores.columns)
print(test1_scores.shape)

NOTE: Cloud Analytic Services made the uploaded file available as table TEST1_SCORES in caslib CASUSER(sasdemo).
NOTE: The table TEST1_SCORES has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.
Index(['Unnamed: 0', 'id', 'cycle', 'nearfailure', '_DL_Pred_', 'NF_PRED'], dtype='object')
(8255, 6)


# ESP Scores

In [12]:
esp_scores = sas.read_csv('/sasdata/pdm/stream_out_score.csv',
                           casout=CASTable(name='esp_scores', replace=True))
print(esp_scores.columns)
print(esp_scores.shape)

NOTE: Cloud Analytic Services made the uploaded file available as table ESP_SCORES in caslib CASUSER(sasdemo).
NOTE: The table ESP_SCORES has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.
Index(['key', 'id', 'cycle', 'P_nearfailure'], dtype='object')
(8162, 4)


# Compare Scores

In [13]:
sas.fedsql.execdirect(
    '''
    create table t3 {options replace=true} as select
        a.id,
        a.cycle,
        a._DL_Pred_ as p_test1,
        c.P_nearfailure as p_esp
    from test1_scores as a
    left join esp_scores as c
    on
        a.id = c.id and
        a.cycle = c.cycle
    '''
)
t3 = sas.CASTable('t3')

NOTE: Table T3 was created in caslib CASUSER(sasdemo) with 8255 rows returned.


In [14]:
t3.head()

,id,cycle,P_TEST1,P_ESP
0,76.0,97.0,0.001030,0.001030
1,76.0,98.0,0.001031,0.001031
2,76.0,99.0,0.001033,0.001033
3,76.0,100.0,0.001035,0.001035
4,76.0,101.0,0.001035,0.001035


In [15]:
sas.fedsql.execdirect(
    '''
    create table t4 {options replace=true} as select
        sum(case when a._DL_Pred_ - c.P_nearfailure > 0.001 then 1 else 0 end) as diff_cnt,
        sum(case when a._DL_Pred_ > 0.5 then 1 else 0 end) as test1_pos,
        sum(case when c.P_nearfailure > 0.5 then 1 else 0 end) as esp_pos
    from test1_scores as a
    left join esp_scores as c
    on
        a.id = c.id and
        a.cycle = c.cycle
    '''
)
t4 = sas.CASTable('t4')

NOTE: Table T4 was created in caslib CASUSER(sasdemo) with 1 rows returned.


In [16]:
t4.head()

,DIFF_CNT,TEST1_POS,ESP_POS
0,0,316,316


# Difference

In [17]:
t3.query('P_TEST1 - P_ESP > 0.001').fetch()

,id,cycle,P_TEST1,P_ESP


In [19]:
sas.fedsql.execdirect(
    '''
    create table t5 {options replace=true} as select
        a.id,
        min(a.cycle) as test1_first,
        min(c.cycle) as esp_first
    from (select * from test1_scores where _DL_Pred_ > 0.5) as a
    left join (select * from esp_scores where P_nearfailure > 0.5) as c
    on
        a.id = c.id and
        a.cycle = c.cycle
    group by a.id
    '''
)
t5 = sas.CASTable('t5')

NOTE: Table T5 was created in caslib CASUSER(sasdemo) with 25 rows returned.


In [20]:
t5.head(1000)

,id,TEST1_FIRST,ESP_FIRST
0,31.0,169.0,169.0
1,56.0,126.0,126.0
2,42.0,141.0,141.0
3,68.0,166.0,166.0
4,41.0,121.0,121.0
5,24.0,174.0,174.0
6,37.0,119.0,119.0
7,40.0,128.0,128.0
8,66.0,130.0,130.0
9,82.0,149.0,149.0
